<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/36-pr%C3%A4sentation-fortgeschrittene-transaktionen/Kopie_von_03_Transformationen_fuer_Fortgeschrittene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Einlesen der Daten 

In [ ]:
# Bibliotheken einmalig installieren

!pip install pyspark pandas pysqlite3 

In [ ]:
!wget -O "https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py"

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [ ]:
import sqlite3
import pandas as pd
import json
import pyspark.sql.functions as pysparkFunctions 
from pyspark.sql.functions import col
os.system("python3 create_data.py")

512

In [ ]:
## Datenbankdaten in Dataframe schreiben
connection_obj = sqlite3.connect('Diddly_Squat_Farm.db')
cursor_obj = connection_obj.cursor()

# Tabelle Fields ausgeben
for row in cursor_obj.execute('SELECT * FROM fields'):
        print(row)

# Tabelle Fields in Dataframe df_fields schreiben
df_fields = pd.read_sql_query("SELECT * FROM fields", connection_obj)
print('\n\nDies ist das erzeugte Dataframe:\n\n', df_fields)

connection_obj.close()

(5, 'Barn Ground', 10)
(6, 'Bank', 10)
(7, 'Far Brossler', 20)


Dies ist das erzeugte Dataframe:

    field_id    field_name  area_in_sqm
0         5   Barn Ground           10
1         6          Bank           10
2         7  Far Brossler           20


In [ ]:
## (Py)Spark starten

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
#DataFrame für Datenbank erzeugen
df_fields= spark.createDataFrame(df_fields) 
df_fields.printSchema()
df_fields.show()

root
 |-- field_id: long (nullable = true)
 |-- field_name: string (nullable = true)
 |-- area_in_sqm: long (nullable = true)

+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
+--------+------------+-----------+



In [ ]:
## JSON-/CSV-Dateien in Dataframes schreiben
df_fru = spark.read.options(header='True', multiline='True', inferSchema='True').csv('Fruits.csv')
df_veg = spark.read.option("multiline",True).json('Vegetables.json')

# erstellte Dataframes ausgeben (10 Einträge ausgeben)
df_fru.show(10)
df_veg.show(10)

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
+------------+-----+----+-----------------+-------+-------------+
only showing top 10 rows

+--------+-----+-------+-----------------+----+---

In [ ]:
df_veg_correct_revenue = df_veg.withColumn("revenue", pysparkFunctions.column("revenue.net"))
df_veg_correct_revenue.show(5)

df_fru_veg = df_fru.union(df_veg_correct_revenue)
df_fru_veg.show()

+--------+-----+-------+-----------------+----+-------------+
|    crop|field|revenue|water_consumption|week|yield_per_sqm|
+--------+-----+-------+-----------------+----+-------------+
|zucchini|    5|      0|                6|   1|            0|
|  radish|    6|      0|                4|   1|            0|
|zucchini|    5|      0|                5|   2|            0|
|  radish|    6|      0|                4|   2|            0|
|zucchini|    5|      0|                6|   3|            0|
+--------+-----+-------+-----------------+----+-------------+
only showing top 5 rows

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7| 

In [ ]:
df_fru_veg = df_fru_veg.dropDuplicates()
df_fru_veg.show(20)

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|  11|             null|    150|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|  10|               10|    100|           25|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   1|               12|      0|            0|
|        null| null|null|             null|   null|         null|
+---------

# Transformationen für Fortgeschrittene

Damit Jeremy sich weitere Inforamtionen aus den Daten beschaffen kann,muss er sich erneut einen Überblick verschaffen. 

In [ ]:
df_fields.show(10)
df_fru_veg.show(30)


+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
+--------+------------+-----------+

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7| 

Für die kommenden Wochen ist mit erhöhten Niederschlägen zu rechnen. Um zuwissen wann es sich lohnt die Felder zu bewässern, will sich Jeremy einen Überblick über die Durchschnittliche Wassermenge pro Woche verschaffen.

In [ ]:

df_alles_was_angebaut_wird = df_fru_veg.groupBy("week").agg(func.avg("water_consumption"))
df_alles_was_angebaut_wird.show()


+----+----------------------+
|week|avg(water_consumption)|
+----+----------------------+
|   7|                  20.0|
|   6|                  18.0|
|   9|                  24.0|
|   5|                  14.0|
|   1|                  12.0|
|  10|                  10.0|
|   3|                  12.0|
|   8|                  17.5|
|  11|                  null|
|   2|                  10.0|
|   4|                  14.0|
|   0|     5.454545454545454|
|  50|                   8.0|
|  32|                   8.0|
|  40|                  10.0|
|  16|                  10.0|
+----+----------------------+



In [ ]:
#Gesamter Wasserverbrauch pro Pflanze aufsummieren Beispiel GroupBy
gesamt_verbrauch =  df_fru_veg.groupBy("crop").agg(func.sum("water_consumption"))
gesamt_verbrauch.show()

+------------+----------------------+
|        crop|sum(water_consumption)|
+------------+----------------------+
|strawberries|                   194|
|    zucchini|                    80|
|      radish|                    45|
+------------+----------------------+



als nächstes Folgen die Beispiel für die Join Bedingung

In [ ]:
#Beispiel JOIN
#Nun will Jeremy wieviele Pflanzen er auf jedem Feld anbauen kann  

gesamt_join = df_fru_veg.join(df_fields, df_fru_veg.field ==  df_fields.field_id,"inner")
gesamt_join.show(30)

+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|field_id|  field_name|area_in_sqm|
+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+
|    zucchini|    5|  40|             null|     11|            0|       5| Barn Ground|         10|
|    zucchini|    5|  40|                5|     10|           25|       5| Barn Ground|         10|
|    zucchini|    5|  40|               12|      9|           25|       5| Barn Ground|         10|
|    zucchini|    5|  40|               13|      8|           25|       5| Barn Ground|         10|
|    zucchini|    5|  16|               10|      7|           25|       5| Barn Ground|         10|
|    zucchini|    5|   8|                9|      6|           10|       5| Barn Ground|         10|
|    zucchini|    5|   0|                7|      5|            5|       5| Barn Ground|         10|


In [ ]:

gesamt_join = gesamt_join.withColumn("yield_per_area", col("yield_per_sqm") * col("area_in_sqm"))
gesamt_join.show()


+--------+-----+----+-----------------+-------+-------------+--------+-----------+-----------+--------------+
|    crop|field|week|water_consumption|revenue|yield_per_sqm|field_id| field_name|area_in_sqm|yield_per_area|
+--------+-----+----+-----------------+-------+-------------+--------+-----------+-----------+--------------+
|zucchini|    5|  40|             null|     11|            0|       5|Barn Ground|         10|             0|
|zucchini|    5|  40|                5|     10|           25|       5|Barn Ground|         10|           250|
|zucchini|    5|  40|               12|      9|           25|       5|Barn Ground|         10|           250|
|zucchini|    5|  40|               13|      8|           25|       5|Barn Ground|         10|           250|
|zucchini|    5|  16|               10|      7|           25|       5|Barn Ground|         10|           250|
|zucchini|    5|   8|                9|      6|           10|       5|Barn Ground|         10|           100|
|zucchini|

In [ ]:
#Nachdem der Ertrag für die einzelnen Gemüse- oder Obstsorten ermittelt wurde müssen wir noch unseren Netto gewinn für die einzelnen Sachen berechenen.
gesamt_ertrag =  gesamt_join.groupBy("crop").agg(func.sum("yield_per_area"), func.sum("water_consumption"))
gesamt_ertrag = gesamt_ertrag.select(col("crop").alias("Crop"),col("sum(water_consumption)").alias("water_consumption"), col("sum(yield_per_area)").alias("yield_per_area"))
gesamt_ertrag.show()


+------------+-----------------+--------------+
|        Crop|water_consumption|yield_per_area|
+------------+-----------------+--------------+
|    zucchini|               80|          1150|
|strawberries|              194|          2300|
|      radish|               45|          4000|
+------------+-----------------+--------------+



Hierfür benötogen wir die zuvor ermittelten Liter je Sorte welche wir mit dem Wert von 20Cent verrechnen müssen um die gesanten Bewässerungskosten zu erhalten.

In [ ]:
gesamt_ertrag = gesamt_ertrag.withColumn("netto_yield_per_area",col("yield_per_area") - (col("water_consumption")* 0.2))
df3 = gesamt_ertrag.withColumn("netto_yield_per_area", func.round(gesamt_ertrag["netto_yield_per_area"], 2))
df3.show()

+------------+-----------------+--------------+--------------------+
|        Crop|water_consumption|yield_per_area|netto_yield_per_area|
+------------+-----------------+--------------+--------------------+
|    zucchini|               80|          1150|              1134.0|
|strawberries|              194|          2300|              2261.2|
|      radish|               45|          4000|              3991.0|
+------------+-----------------+--------------+--------------------+



In [ ]:
df3.orderBy(col("netto_yield_per_area").desc()).show(truncate=False)



+------------+-----------------+--------------+--------------------+
|Crop        |water_consumption|yield_per_area|netto_yield_per_area|
+------------+-----------------+--------------+--------------------+
|radish      |45               |4000          |3991.0              |
|strawberries|194              |2300          |2261.2              |
|zucchini    |80               |1150          |1134.0              |
+------------+-----------------+--------------+--------------------+



Nun mach ich noch irgendwas anderes und dann sollte das wenigstens für heute passen 